In [64]:

extracted_page_data = {

# "source": "/Users/pratikraj/Desktop/Floor_Plan_Reader/pdf_images/10cbd17d-272c-4929-955b-8fe0056d5c9f-04.jpg",
"extracted_properties": {
"Flat Unit": "X02",
"No Of Bedrooms": "1",
"Dimension": [
    {
    "name": "BALCONY",
    "dimension": "3.09 X 1.79"
    },
    {
    "name": "LIVING ROOM",
    "dimension": "7.540 X 3.87"
    },
    {
    "name": "KITCHEN",
    "dimension": "4.10 X 2.40"
    },
    {
    "name": "BEDROOM",
    "dimension": "4.38 X 3.40"
    },
    {
    "name": "Bedroom 1",
     "dimension": "38 X 3.40"
    },
    {
    "name": "Bed room 1",
     "dimension": "4 X 3"
    },
    {
    "name": "BATHROOM",
    "dimension": "2.50 X 1.60"
    },
    {
    "name": "BALCONY",
    "dimension": "6.10 X 5.17"
    }
],
"Total Area": [
    {
    "name": "SUITE AREA",
    "dimension": "64.17 SQM. 691 SQFT."
    },
    {
    "name": "BALCONY AREA",
    "dimension": "17.73 SQ.M. 191 SQFT"
    },
    {
    "name": "TOTAL AREA",
    "dimension": "81.90 SQM. 882 SQFT."
    }
]
}
}


In [44]:

# # extracted_page_data = extracted_document[i].metadata
# room_data={}
# for dimension in extracted_page_data['extracted_properties']['Dimension']:
#     if(room_data.get(dimension['name'],None) is not None):
#         room_data[dimension['name']][extracted_page_data['extracted_properties']['Flat Unit']]=dimension["dimension"]
#     else:
#         room_data[dimension['name']]={
#             extracted_page_data['extracted_properties']['Flat Unit']:dimension["dimension"]
#         }

# print(room_data)

In [45]:
# {
#     Bedroom:{
#         X02:{
#             Bedroom1:100,
#             Bedroom2:200
#         }
#     }
# }

In [56]:
# # extracted_page_data = extracted_document[i].metadata
# room_data={}

# for dimension in extracted_page_data['extracted_properties']['Dimension']:
#     room_name = dimension['name']
#     room_dimension = dimension['dimension']
#     flat_unit = extracted_page_data['extracted_properties']['Flat Unit']
    
#     if room_name not in room_data:
#         room_data[room_name] = {flat_unit: {f"{room_name}1": room_dimension}}
#     else:
#         if flat_unit not in room_data[room_name]:
#             # room_data[room_name][flat_unit] = {f"{room_name}1": room_dimension}
#         else:
#             num = len([key for key in room_data[room_name][flat_unit] if room_name in key]) + 1
#             room_data[room_name][flat_unit][f"{room_name}{num}"] = room_dimension

# print(room_data)


{'BALCONY': {'X02': {'BALCONY1': '3.09 X 1.79', 'BALCONY2': '6.10 X 5.17'}}, 'LIVING ROOM': {'X02': {'LIVING ROOM1': '7.540 X 3.87'}}, 'KITCHEN': {'X02': {'KITCHEN1': '4.10 X 2.40'}}, 'BEDROOM': {'X02': {'BEDROOM1': '4.38 X 3.40'}}, 'Bedroom 1': {'X02': {'Bedroom 11': '38 X 3.40'}}, 'Bed room 1': {'X02': {'Bed room 11': '4 X 3'}}, 'BATHROOM': {'X02': {'BATHROOM1': '2.50 X 1.60'}}}


In [47]:


# import re
# def restructure_room_name(room_name):
#     # Convert room name to lowercase, remove spaces, and remove numbers
#     restructured_name = room_name.lower().replace(" ", "")
#     restructured_name = re.sub(r'\d+', '', restructured_name)
#     return restructured_name

# restructure_room_name('Bed room1')

In [65]:
import re
def restructure_room_name(room_name):
    restructured_name = room_name.lower().replace(" ", "")
    restructured_name = re.sub(r'\d+', '', restructured_name)
    return restructured_name

extracted_page_data = extracted_document[i].metadata
room_data = {}

for dimension in extracted_page_data['extracted_properties']['Dimension']:
    room_name=restructure_room_name(dimension['name'])
    room_dimension = dimension['dimension']
    flat_unit = extracted_page_data['extracted_properties']['Flat Unit']
    
    if room_name not in room_data:
        room_data[room_name] = {flat_unit: {f"{room_name}1": room_dimension}}
    else:
        if flat_unit not in room_data[room_name]:
            room_data[room_name][flat_unit] = {f"{room_name}1": room_dimension}
        else:
            num = len([key for key in room_data[room_name][flat_unit] if room_name in key]) + 1
            room_data[room_name][flat_unit][f"{room_name}{num}"] = room_dimension
            


print(room_data)



{'balcony': {'X02': {'balcony1': '3.09 X 1.79', 'balcony2': '6.10 X 5.17'}}, 'livingroom': {'X02': {'livingroom1': '7.540 X 3.87'}}, 'kitchen': {'X02': {'kitchen1': '4.10 X 2.40'}}, 'bedroom': {'X02': {'bedroom1': '4.38 X 3.40', 'bedroom2': '38 X 3.40', 'bedroom3': '4 X 3'}}, 'bathroom': {'X02': {'bathroom1': '2.50 X 1.60'}}}


In [76]:
from langchain.document_loaders import JSONLoader
from langchain.schema import Document


num_elements = len(room_data)
page_nos = range(1, num_elements + 1)
documents = [create_document(room_data, page_no) for page_no in page_nos]

def create_document(room_data, page_no):
   document_content = ''
   prev_room_name = None
   for room_name, room_info in room_data.items():
       for flat_unit, room_dimensions in room_info.items():
           for dimension_key, room_dimension in room_dimensions.items():
               if room_name == prev_room_name:
                  # If the room name is the same as the previous room name, append the new dimension to the existing line
                  document_content += f'{dimension_key}={room_dimension} '
               else:
                  if prev_room_name is not None:
                      # If there was a previous room name, add a newline before starting a new line
                      document_content += '\n'
                  document_content += f'{room_name}: {flat_unit}: {dimension_key}={room_dimension} '
               prev_room_name = room_name
   document_content += f'Page: {page_no}'
   return Document(page_content=document_content)

for document in documents:
   print(document.page_content)


balcony: X02: balcony1=3.09 X 1.79 balcony2=6.10 X 5.17 
livingroom: X02: livingroom1=7.540 X 3.87 
kitchen: X02: kitchen1=4.10 X 2.40 
bedroom: X02: bedroom1=4.38 X 3.40 bedroom2=38 X 3.40 bedroom3=4 X 3 
bathroom: X02: bathroom1=2.50 X 1.60 Page: 1
balcony: X02: balcony1=3.09 X 1.79 balcony2=6.10 X 5.17 
livingroom: X02: livingroom1=7.540 X 3.87 
kitchen: X02: kitchen1=4.10 X 2.40 
bedroom: X02: bedroom1=4.38 X 3.40 bedroom2=38 X 3.40 bedroom3=4 X 3 
bathroom: X02: bathroom1=2.50 X 1.60 Page: 2
balcony: X02: balcony1=3.09 X 1.79 balcony2=6.10 X 5.17 
livingroom: X02: livingroom1=7.540 X 3.87 
kitchen: X02: kitchen1=4.10 X 2.40 
bedroom: X02: bedroom1=4.38 X 3.40 bedroom2=38 X 3.40 bedroom3=4 X 3 
bathroom: X02: bathroom1=2.50 X 1.60 Page: 3
balcony: X02: balcony1=3.09 X 1.79 balcony2=6.10 X 5.17 
livingroom: X02: livingroom1=7.540 X 3.87 
kitchen: X02: kitchen1=4.10 X 2.40 
bedroom: X02: bedroom1=4.38 X 3.40 bedroom2=38 X 3.40 bedroom3=4 X 3 
bathroom: X02: bathroom1=2.50 X 1.60 Page

In [ ]:
!pip install langchain